# Importing Libraries

in this part we will install all the necessary libraries on command prompt and then import the necessary functions from those libraries. 

In [1]:
# importing all the necessary libraries

import pandas as pd



from numpy import mean

import numpy as np

import time



# step 1: preprocessing

from sklearn.impute import SimpleImputer # import some strategic imputer to fill in any missing values using mean

from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler, Normalizer # scale all the values to one range to avoid any biasness (this bias is seen in mostly naive bayes and knn etc)



from sklearn.impute import KNNImputer # import some strategic imputer to fill missing values using KNN (finds the nearest neighbour and fills it with that value)



from sklearn.feature_selection import SequentialFeatureSelector, SelectKBest, f_classif, VarianceThreshold

from sklearn.feature_selection import f_regression

from sklearn.decomposition import PCA

from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder

from sklearn.pipeline import Pipeline

from sklearn.linear_model import Ridge, Lasso, LinearRegression



# step 2: data division

from sklearn.model_selection import train_test_split, RepeatedKFold, cross_val_score, GridSearchCV, ParameterGrid # to divide the code into train/test using a specific percentage or with/without replacement



# step 3: model

from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, StackingRegressor



# step 4: displaying accuracy

from sklearn.metrics import roc_auc_score, accuracy_score # to display the accuracy of our tree

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, make_scorer



# step 5: warning filter

import warnings

warnings.filterwarnings('ignore')

In [2]:
from datetime import datetime



# Function to get current date and time as a string

def get_current_datetime():

    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Data Loading

data shall be loaded into variables as data sets using pandas and csv readers. they will be checked to see if they are loaded properly and will be loaded as 2 sets: train and test as per given in the kaggle data

In [3]:
# lets load the training data set

# train_data = pd.read_csv(r"/kaggle/input/challenge-2/train/train.csv")

train_data = pd.read_csv(r"/kaggle/input/challenge2/train.csv")

# lets also check it by getting the first few rows of the data, there should be x1 - x78 and one target variable Y

train_data.head() 

,full_sq,life_sq,floor,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,43.0,27.0,4.0,Investment,Bibirevo,6407578.100,155572.0,0.189727,0.000070,9576.0,...,9.0,4.0,0.0,13.0,22.0,1.0,0.0,52.0,4.0,5850000.0
1,34.0,19.0,3.0,Investment,Nagatinskij Zaton,9589336.912,115352.0,0.372602,0.049637,6880.0,...,15.0,3.0,0.0,15.0,29.0,1.0,10.0,66.0,14.0,6000000.0
2,43.0,29.0,2.0,Investment,Tekstil'shhiki,4808269.831,101708.0,0.112560,0.118537,5879.0,...,10.0,3.0,0.0,11.0,27.0,0.0,4.0,67.0,10.0,5700000.0
3,77.0,77.0,4.0,Investment,Basmannoe,8398460.622,108171.0,0.015234,0.037316,5706.0,...,319.0,108.0,17.0,135.0,236.0,2.0,91.0,195.0,14.0,16331452.0
4,67.0,46.0,14.0,Investment,Nizhegorodskoe,7506452.020,43795.0,0.007670,0.486246,2418.0,...,62.0,14.0,1.0,53.0,78.0,1.0,20.0,113.0,17.0,9100000.0


In [4]:
# lets load the test data

# test_data = pd.read_csv(r"/kaggle/input/challenge-2/test/test.csv")

test_data = pd.read_csv(r"/kaggle/input/challenge2/test.csv")

# check if the data has been loaded by getting the first 5 rows - there should be x1 - x78 and no target variable Y as this is test data

test_data.head() 

,row ID,full_sq,life_sq,floor,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,...,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000
0,Row3,89.0,50.0,9.0,Investment,Mitino,1.258354e+07,178473.0,0.194703,0.069753,...,15.0,11.0,2.0,1.0,4.0,4.0,0.0,0.0,26.0,3.0
1,Row6,25.0,14.0,10.0,Investment,Sokol'niki,1.032047e+07,57405.0,0.523439,0.042307,...,144.0,81.0,16.0,3.0,38.0,80.0,1.0,27.0,127.0,8.0
2,Row11,38.0,19.0,11.0,Investment,Zapadnoe Degunino,7.632940e+06,78810.0,0.051844,0.437885,...,39.0,8.0,3.0,0.0,10.0,9.0,0.0,0.0,35.0,4.0
3,Row12,43.0,28.0,4.0,Investment,Kuncevo,5.235177e+07,142462.0,0.070662,0.035145,...,21.0,13.0,9.0,1.0,7.0,15.0,0.0,2.0,47.0,0.0
4,Row14,31.0,21.0,3.0,Investment,Lefortovo,8.993640e+06,89971.0,0.066941,0.306977,...,205.0,88.0,19.0,2.0,63.0,100.0,0.0,28.0,132.0,14.0


# Data Preprocessing

before we start processing this data and using algorithms, we will fix this data first, this is called data preprocessing

## split data into categorical and numerical

categorical will have one-hot and simple imputer of most frequent while numerical will have simple mean imputer and minmax scaler

In [5]:
categorical_cols = train_data.select_dtypes(include=["object"]).columns

numerical_cols = train_data.select_dtypes(exclude=["object"]).drop(columns=['price_doc']).columns

In [6]:
num_transformer = Pipeline(steps=[

    ("imputer", SimpleImputer(strategy="median")),

    ("scaler", StandardScaler())

])



cat_transformer = Pipeline(steps=[

    ("imputer", SimpleImputer(strategy="most_frequent")),

    ("onehot", OneHotEncoder(handle_unknown="ignore"))

])

In [7]:
# Column transformer for preprocessing

preprocessor = ColumnTransformer(

    transformers=[

        ("num", num_transformer, numerical_cols), 

        ("cat", cat_transformer, categorical_cols)

    ]

)

## correlation matrix

i tried getting the correlation matrix but apparently a 2000 columns matrix is very computationally expensive as it performs pairs for all. so dont run it. it takes too long and then fails. i ran for 5 minutes. 

In [8]:
# # DONT RUN

# corr_matrix = train_data.corr()

# print(corr_matrix)

# PCA

principal component analysis is applied

In [9]:
# # -------------------------- case  --------------------------

# pca = PCA(n_components=33)                                 

# X = pca.fit_transform(X)

# test_data_processed = pca.transform(test_data_processed)

## Data Splitting - festures and targets

the data in train_data set is of x1 - x78 columns (79 variables) and one target variable (Y). we must split that data so that we can perform data preprocessing on the features variables (will be referred to as X).

In [10]:
# so in X, it is ALL the columns EXCEPT the last column known as 'Y' (we can confirm this using the train_data.head() we did earlier) so we must get all columns and DROP only the 'y' column

X = train_data.drop(columns=['price_doc'])

X # lets display X and see what it is now

,full_sq,life_sq,floor,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,...,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000
0,43.0,27.0,4.0,Investment,Bibirevo,6.407578e+06,155572.0,0.189727,0.000070,9576.0,...,40.0,9.0,4.0,0.0,13.0,22.0,1.0,0.0,52.0,4.0
1,34.0,19.0,3.0,Investment,Nagatinskij Zaton,9.589337e+06,115352.0,0.372602,0.049637,6880.0,...,36.0,15.0,3.0,0.0,15.0,29.0,1.0,10.0,66.0,14.0
2,43.0,29.0,2.0,Investment,Tekstil'shhiki,4.808270e+06,101708.0,0.112560,0.118537,5879.0,...,25.0,10.0,3.0,0.0,11.0,27.0,0.0,4.0,67.0,10.0
3,77.0,77.0,4.0,Investment,Basmannoe,8.398461e+06,108171.0,0.015234,0.037316,5706.0,...,552.0,319.0,108.0,17.0,135.0,236.0,2.0,91.0,195.0,14.0
4,67.0,46.0,14.0,Investment,Nizhegorodskoe,7.506452e+06,43795.0,0.007670,0.486246,2418.0,...,155.0,62.0,14.0,1.0,53.0,78.0,1.0,20.0,113.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181502,48.0,33.0,3.0,Investment,Poselenie Mihajlovo-Jarcevskoe,6.455617e+07,4949.0,0.586175,0.005819,346.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
181503,48.0,33.0,3.0,Investment,Poselenie Mihajlovo-Jarcevskoe,6.455617e+07,4949.0,0.586175,0.005819,346.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
181504,48.0,33.0,3.0,Investment,Poselenie Mihajlovo-Jarcevskoe,6.455617e+07,4949.0,0.586175,0.005819,346.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
181505,48.0,33.0,3.0,Investment,Poselenie Mihajlovo-Jarcevskoe,6.455617e+07,4949.0,0.586175,0.005819,346.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0


In [11]:
# so as per our X output, we can see that number of columns in train_data is 79 and number of columns in X is 78 meaning we have successfully performed our removal of target variable

# now to get the target variable alone, we can just get it alone,

Y = train_data['price_doc']

Y # lets see what it is

# as per our Y output, we can see it is of one column and 246k rows which means we have successfully extracted the target variable column

0          5850000.0
1          6000000.0
2          5700000.0
3         16331452.0
4          9100000.0
             ...    
181502     3480000.0
181503     3480000.0
181504     3480000.0
181505     3480000.0
181506     3480000.0
Name: price_doc, Length: 181507, dtype: float64

# Filters

there are two types of filters to filter out columns/features:

- variance filter (a column which has same values throughout the column like all are sunny)

- correlation filter (two columns which are same like weight in kg and weight in pounds)

In [12]:
# print("X : ", X.shape)

# print("test data : ", test_data_processed.shape)

In [13]:
# variance filter

# ----------------------------- case  -----------------------------

# variance_filter = VarianceThreshold(threshold=0.001)  # Adjust the threshold if needed

# X = variance_filter.fit_transform(X)

# test_data_processed = variance_filter.fit_transform(test_data_processed)

X.shape

(181507, 271)

In [14]:
# test_data_processed.shape

In [15]:
# # correlation filter

# # ----------------------------- case  -----------------------------

# corr_matrix = pd.DataFrame(X).corr().abs()

# upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.9)]

# X = pd.DataFrame(X).drop(columns=to_drop)

# test_data_processed = pd.DataFrame(test_data_processed).drop(columns=to_drop)

X.shape

(181507, 271)

In [16]:
# test_data_processed.shape

## Data Splitting - train and validate

now our test_data set is of rows with NO target variable whereas the train_data set is WITH target variable.

our rules in machine learning is that we must train half or 70% of the data and then we must check its accuracy using the remaining half or 30% of the data - we can only check accuracy IF we have the answers i.e. the target variable. 

So, what we need to do is, is split the train_data set into 2, by a 70% and 30% ratio. we train the model using the 70% and then test the model using the 30% and then use that model to predict the test_data set.

In [17]:
# holdout method

trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.3, random_state=2)

# functions

here we have defined functions like forward-backward selection, kbest selection & algorithm feature importance

In [18]:
# forward backward selection

def fbselection(direction, sample_model, features, X, trainX, trainY, testX, test_data_processed):

    print("starting")

    selection = SequentialFeatureSelector(sample_model, direction=direction, n_features_to_select=features, scoring='roc_auc')

    return modelSelector(sample_model, selection, X, trainX, trainY, testX, test_data_processed)



def modelSelector(sample_model, selection, X, trainX, trainY, testX, test_data_processed):

    print("start extracting")

    trainX = selection.fit_transform(trainX, trainY)

    print("extracted, transforming")

    testX = selection.transform(testX)                                  # Ensure the test set is transformed similarly

    test_data_processed = selection.transform(test_data_processed)      # test data is also transformed

    X = selection.transform(X)                                          # full data transforming

    print("transformed")

    return sample_model, X, trainX, trainY, testX, test_data_processed



# kbest selection

def kbest(sample_model, features, X, trainX, trainY, testX, test_data_processed):

    print("starting")

    selection = SelectKBest(score_func=f_regression, k=features)

    return modelSelector(sample_model, selection, X, trainX, trainY, testX, test_data_processed)

In [19]:
# feature importance function

def featureImportance(sample_model, features, X, trainX, trainY, testX, test_data_processed):

    print("fitting")

    

    # fit the model

    sample_model.fit(trainX, trainY)



    print("extracting features")



    # extract all the feature names from data

    importances = sample_model.feature_importances_

    feature_names = train_data.drop(columns=['price_doc']).columns

    print(feature_names)



    # sort with respect to importance

    feature_importance_df = pd.DataFrame({

        'Feature': feature_names,

        'Importance': importances

    }).sort_values(by='Importance', ascending=False)



    # extract the top ones

    top_features = feature_importance_df['Feature'].head(features).values

    print(top_features)



    # change all data according to the top ones we have selected

    trainX = pd.DataFrame(trainX, columns=feature_names)[top_features]

    testX = pd.DataFrame(testX, columns=feature_names)[top_features]

    X = pd.DataFrame(X, columns=feature_names)[top_features]

    test_data_processed = pd.DataFrame(test_data_processed, columns=feature_names)[top_features]



    print("features extracted")

    

    # retrain the model

    sample_model.fit(trainX, trainY)



    print("features trained")

    

    return sample_model, X, trainX, trainY, testX, test_data_processed

## model intialization

here model is intialized

In [20]:
# declare the model here

# Define base estimators
rf1 = Ridge(alpha=100, solver='lsqr', tol=0.001, max_iter=1000)

dt2 = Lasso(alpha=10000, max_iter=1000)

xgb = LinearRegression(n_jobs=-1)

# Define meta-regressor
meta_regressor = RandomForestRegressor(
    max_depth=32,
    n_estimators=1500,
    max_features='log2',
    min_samples_leaf=2,
    min_samples_split=3,
    bootstrap=True,
    verbose=2,
    n_jobs=-1
)

# Create stacking regressor
stacking = StackingRegressor(
    estimators=[('rf1', rf1), ('dt2', dt2), ('xgb1', xgb)],
    final_estimator=meta_regressor,
    passthrough=False, n_jobs=-1, verbose=2
)



model = Pipeline(steps=[

    ("preprocessor", preprocessor),

    ("model", stacking)

])

In [21]:
print("X shape -> ", X.shape)

print("trainX shape -> ", trainX.shape)

print("testX shape -> ", testX.shape)

print("test_data_processed shape -> ", test_data.shape)

X shape ->  (181507, 271)
trainX shape ->  (127054, 271)
testX shape ->  (54453, 271)
test_data_processed shape ->  (77789, 272)


# feature selection

here we will apply feature selection and feature importance

In [22]:
# Feature selection

# model, X, trainX, trainY, testX, test_data = featureImportance(

#     model, 40, X, trainX, trainY, testX, test_data

# )

In [23]:
print("X shape -> ", X.shape)

print("trainX shape -> ", trainX.shape)

print("testX shape -> ", testX.shape)

print("test_data_processed shape -> ", test_data.shape)

X shape ->  (181507, 271)
trainX shape ->  (127054, 271)
testX shape ->  (54453, 271)
test_data_processed shape ->  (77789, 272)


# grid search

In [24]:
def gridsearch(param_grid, model, scorer, trainX, trainY):

    print("starting grid search")



    # intialize grid search

    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring=scorer, verbose=3)

    print("grid search is intialized")



    # fit the model

    grid_search.fit(trainX, trainY)

    print("grid search fitting completed")



    # display the best model grid search found

    best_model = grid_search.best_estimator_

    print(best_model)



    # display the best parameters of the best model

    best_parameters = grid_search.best_params_

    print(best_parameters)



    # display the best score of the best model

    print("Best cross-validated score:", grid_search.best_score_)



    # assign the best model our model

    model = best_model

    print("model assigned, grid search completed")



    return model

In [25]:
# # Define a scoring metric (e.g., negative mean squared error)

# scorer = make_scorer(mean_squared_error, greater_is_better=False)

In [26]:
# # define hyper parameters of grid

# param_grid = {

#     'estimators': [10, 100, 200, 500, 1000, 2000, 3000], 

#     'learning_rate': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.5],

#     'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# }

In [27]:
# sample from train_data

# sample_train = train_data.sample(frac=0.5)

# sample_X = sample_train.drop('price_doc', axis=1) 

# sample_Y = sample_train['price_doc']

In [28]:
# model = gridsearch(param_grid, model, scorer, trainX, trainY)

## model running

here we run the model

In [29]:
from datetime import datetime



def get_current_datetime():

    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [30]:
# compute this predictions metrics

def metrics(y_pred, testY):

    print("starting to compute metrics")

    

    # display the mean squared error of this prediction

    mse = mean_squared_error(testY, y_pred)

    print("Mean squared error: %.2f" % mse, "   ")



    # display the root mean squared error

    rmse = np.sqrt(mse)  # Root Mean Squared Error

    print("Root Mean squared error: %.2f" % rmse, "   ")



    # display the mean absolute error of this prediction

    mae = mean_absolute_error(testY, y_pred)

    print("Mean absolute error: %.2f" % mae, "   ")



    # display the coeffeicient of determination of this preduction

    r2_Score = r2_score(testY, y_pred)

    print("Coefficient of determination: %.2f" % r2_Score, "    ")

    

    return rmse

In [31]:
def run_model(model, trainX, trainY, testX, testY):

    print("training model ", get_current_datetime())

    model.fit(trainX, trainY)

    print("computing score ", get_current_datetime())

    print("model score: ", model.score(trainX, trainY) , get_current_datetime())

    y_pred = model.predict(testX)

    rmse = metrics(y_pred, testY)

    return model, rmse

In [32]:
def createFile(model, X, Y, test_data, file_name):

    print("fitting on X Y ", get_current_datetime())

    model.fit(X, Y)



    print("scoring on X Y ", get_current_datetime())

    score = model.score(X, Y)

    print("model test score: ", score, "    ")



    print("predicting on test ", get_current_datetime())

    test_prediction = model.predict(test_data)

    print(test_prediction)



    print("getting sample submission ", get_current_datetime())

    sample_data = pd.read_csv(r"/kaggle/input/challenge2/sample_submission.csv")

    sample_data['price_doc'] = test_prediction



    print("Saving submission ", get_current_datetime())

    # Ensure the path ends with a backslash

    base_path = r"/kaggle/working/"

    full_path = base_path + file_name

    sample_data.to_csv(full_path, index=False)

    print(f"File saved at: {full_path}")

In [33]:
def create_submission(model, trainX, trainY, testX, testY, X, Y, test_data, file_name):

    model, rmse = run_model(model, trainX, trainY, testX, testY)

    createFile(model, X, Y, test_data, file_name)

In [34]:
create_submission(model, trainX, trainY, testX, testY, X, Y, test_data, "stacking1.csv")

training model  2024-11-29 22:14:08


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.003873463261962e+18, tolerance: 6044579044802088.0
  model = cd_fast.sparse_enet_coordinate_descent(
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   20.3s finished
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  4.5min finished
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.0119881081443645e+18, tolerance: 4838307773164084.0
  model = cd_fast.sparse_enet_coordinate_descent(
/o

building tree 1 of 1500
building tree 2 of 1500
building tree 3 of 1500
building tree 4 of 1500
building tree 5 of 1500
building tree 6 of 1500
building tree 7 of 1500
building tree 8 of 1500
building tree 9 of 1500
building tree 10 of 1500
building tree 11 of 1500
building tree 12 of 1500
building tree 13 of 1500
building tree 14 of 1500
building tree 15 of 1500
building tree 16 of 1500
building tree 17 of 1500
building tree 18 of 1500
building tree 19 of 1500
building tree 20 of 1500
building tree 21 of 1500
building tree 22 of 1500
building tree 23 of 1500
building tree 24 of 1500
building tree 25 of 1500
building tree 26 of 1500
building tree 27 of 1500
building tree 28 of 1500
building tree 29 of 1500
building tree 30 of 1500
building tree 31 of 1500
building tree 32 of 1500
building tree 33 of 1500
building tree 34 of 1500
building tree 35 of 1500
building tree 36 of 1500
building tree 37 of 1500
building tree 38 of 1500
building tree 39 of 1500
building tree 40 of 1500


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    4.1s


building tree 41 of 1500
building tree 42 of 1500
building tree 43 of 1500
building tree 44 of 1500
building tree 45 of 1500
building tree 46 of 1500
building tree 47 of 1500
building tree 48 of 1500
building tree 49 of 1500
building tree 50 of 1500
building tree 51 of 1500
building tree 52 of 1500
building tree 53 of 1500
building tree 54 of 1500
building tree 55 of 1500
building tree 56 of 1500
building tree 57 of 1500
building tree 58 of 1500
building tree 59 of 1500
building tree 60 of 1500
building tree 61 of 1500
building tree 62 of 1500
building tree 63 of 1500
building tree 64 of 1500
building tree 65 of 1500
building tree 66 of 1500
building tree 67 of 1500
building tree 68 of 1500
building tree 69 of 1500
building tree 70 of 1500
building tree 71 of 1500
building tree 72 of 1500
building tree 73 of 1500
building tree 74 of 1500
building tree 75 of 1500
building tree 76 of 1500
building tree 77 of 1500
building tree 78 of 1500
building tree 79 of 1500
building tree 80 of 1500


...

scoring on X Y  2024-11-29 23:34:28


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    2.0s
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed:    4.7s
[Parallel(n_jobs=4)]: Done 640 tasks      | elapsed:    8.4s
[Parallel(n_jobs=4)]: Done 1005 tasks      | elapsed:   13.7s
[Parallel(n_jobs=4)]: Done 1450 tasks      | elapsed:   19.5s
[Parallel(n_jobs=4)]: Done 1500 out of 1500 | elapsed:   20.1s finished


model test score:  0.6314408654849808     
predicting on test  2024-11-29 23:34:54


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    1.0s
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed:    2.2s
[Parallel(n_jobs=4)]: Done 640 tasks      | elapsed:    3.9s
[Parallel(n_jobs=4)]: Done 1005 tasks      | elapsed:    6.2s
[Parallel(n_jobs=4)]: Done 1450 tasks      | elapsed:    9.0s
[Parallel(n_jobs=4)]: Done 1500 out of 1500 | elapsed:    9.3s finished


[11627059.63699285 13168402.44627124  3937451.30419883 ...
  4614322.85256813  4614322.85256813  4614322.85256813]
getting sample submission  2024-11-29 23:35:05
Saving submission  2024-11-29 23:35:05
File saved at: /kaggle/working/stacking1.csv


In [35]:
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['full_sq', 'life_sq', 'floor', 'area_m', 'raion_popul',
       'green_zone_part', 'indust_part', 'children_preschool',
       'preschool_education_centers_raion', 'children_school',
       ...
       'cafe_count_5000_pri...
      dtype='object'))])),
                ('model',
                 StackingRegressor(estimators=[('rf1',
                                                Ridge(alpha=100, max_iter=1000,
                                                      solver='lsqr',
                                                      tol=0.001)),
                                               ('dt2', Lasso(alpha=10000)),
                                               ('xgb1',
                                                LinearRegression(n_jobs=-1))],
                                   final_estimator=RandomForestRegressor(max_depth=32,
                                                                         max_features='log2',
                                                                         min_samples_leaf=2,
                                                                         min_samples_split=3,
                                                                         n_estimators=1500,
                                                                         n_jobs=-1,
                                                                         verbose=2),
                                   n_jobs=-1, verbose=2))])